In [1]:
# notes
# machine learning works with tables
# rows are examples and columns are atributes
# tensor = n-dimensional array

In [2]:
# in genDL repo
# hi.true = high expression, true sites from highly expressed genes
# lo.true = low expression
# fakes have the same GT/AG site, but everything else is random
# write something that takes the true set, fakes, and a fasta file as input
# fasta is the thousands of examples of true or fake sites
# use ex3 data for testing. ex5 has one replaced nucleotide.
# tttcAG, where tttc can be tttc or cttt in ex3. AG does not change.
# 3 files you want to use: hi.true, lo.true, ex3. trying to identify acceptor sites in ex3
# when labeling data, run only acc or don at a time.
# labels not for acc, don, they are positive/negative
# 1 for positive, 0 for negative

In [3]:
# one-hot encoding
# A: [1,0,0,0]
# C: [0,1,0,0]
# G: [0,0,1,0]
# T: [0,0,0,1]
# https://d2l.ai/chapter_preliminaries/pandas.html#reading-the-dataset


# need to re-do one-hot encoding
# [1,0,0,0,0,1,0,0] is AC
# NOT
# [1,0,0,0][0,1,0,0] is AC

In [4]:
import pandas as pd
import gzip
import torch

def encoder(file):

    fp = gzip.open(file)

    one_hot = {
        'A': [1,0,0,0],
        'C': [0,1,0,0],
        'G': [0,0,1,0],
        'T': [0,0,0,1]
    }

    encoded = []

    for line in fp:
        onehot_seq = []
        bstring = line.rstrip()
        line = bstring.decode()
        if line.startswith('>'): continue
        for n in line:
            onehot_seq.append(one_hot[n])
        encoded.append(onehot_seq)

    return encoded
    
encoded_true = encoder("data/acc.ex3.fa.gz")
encoded_false = encoder("data/acc.fake.txt.fa.gz")

inputs_true = pd.DataFrame(data = encoded_true)
inputs_false = pd.DataFrame(data = encoded_false)

label_true = [1]*len(encoded_true)
label_false = [0]*len(encoded_false)

inputs_true["label"] = label_true
inputs_false["label"] = label_false

encoded_data = [inputs_true, inputs_false]
encoded_data = pd.concat(encoded_data)
# need to convert onehot encoded lists into a tensor?
#torch.from_numpy(encoded_data)

inputs, outputs = encoded_data.iloc[:, 0:42], encoded_data.iloc[:, 42]

print(inputs_true.shape, inputs_false.shape)
encoded_data.shape
print(inputs.values)
#X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)

(10000, 43) (186671, 43)
[[list([1, 0, 0, 0]) list([0, 0, 0, 1]) list([0, 0, 1, 0]) ...
  list([0, 0, 1, 0]) list([1, 0, 0, 0]) list([1, 0, 0, 0])]
 [list([0, 0, 1, 0]) list([0, 1, 0, 0]) list([0, 1, 0, 0]) ...
  list([0, 0, 1, 0]) list([1, 0, 0, 0]) list([0, 0, 1, 0])]
 [list([0, 0, 1, 0]) list([0, 1, 0, 0]) list([0, 1, 0, 0]) ...
  list([0, 0, 0, 1]) list([0, 1, 0, 0]) list([0, 0, 1, 0])]
 ...
 [list([1, 0, 0, 0]) list([1, 0, 0, 0]) list([0, 0, 0, 1]) ...
  list([0, 0, 0, 1]) list([0, 0, 1, 0]) list([1, 0, 0, 0])]
 [list([0, 1, 0, 0]) list([1, 0, 0, 0]) list([0, 0, 1, 0]) ...
  list([0, 0, 1, 0]) list([0, 0, 0, 1]) list([0, 0, 0, 1])]
 [list([0, 0, 1, 0]) list([0, 0, 0, 1]) list([0, 0, 1, 0]) ...
  list([0, 0, 0, 1]) list([0, 0, 0, 1]) list([0, 0, 1, 0])]]


In [5]:
# mix in the reals and fakes for the input
# two different labels
# titanic who lives who dies ML practice

In [6]:
type(inputs.values)

numpy.ndarray

In [7]:
type(encoded_data)

pandas.core.frame.DataFrame

In [8]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # Column names
    f.write('NA,Pave,127500\n')  # Each row represents a data example
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')
    
data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


In [9]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


In [10]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


In [11]:
X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

In [14]:
print(inputs.values)
print(inputs)
type(inputs.values)

[[3. 1. 0.]
 [2. 0. 1.]
 [4. 0. 1.]
 [3. 0. 1.]]
   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


numpy.ndarray

In [16]:
test_values = pd.array([1,0,0,0],[0,0,1,0])
print(test_values)

TypeError: Field elements must be 2- or 3-tuples, got '0'